In [61]:
import pandas as pd
import numpy as np
import matplotlib as plt
from datetime import datetime

races = pd.read_csv('data/races.csv')
qualifying = pd.read_csv('data/qualifying.csv')
constructors = pd.read_csv('data/constructor_standings.csv')
drivers = pd.read_csv('data/driver_standings.csv')
results = pd.read_csv('data/results.csv')
weather = pd.read_csv('data/weather.csv')


In [62]:
qualifying.rename(columns={'grid_pos': 'grid'}, inplace=True)

In [63]:
df1 = pd.merge(races, weather, how='inner', on=['season', 'round', 'circuit_id']).drop(['lat', 'long','country','weather'], axis = 1)
df2 = pd.merge(df1, results, how='inner', on=['season', 'round', 'circuit_id']).drop(['url','points', 'status', 'time'], axis = 1)
df3 = pd.merge(df2, drivers, how='left', on=['season', 'round', 'driver']) 
df4 = pd.merge(df3, constructors, how='left', on=['season', 'round', 'constructor'])

merged_df = pd.merge(df4, qualifying, how='inner', on=['season', 'round', 'grid']).drop(['driver_y', 'constructor_y'], axis=1)
merged_df.rename(columns={'driver_x': 'driver', 'constructor_x': 'constructor'}, inplace=True)

In [64]:
from dateutil.relativedelta import *

merged_df['date'] = pd.to_datetime(merged_df.date)
merged_df['date_of_birth'] = pd.to_datetime(merged_df.date_of_birth)
merged_df['driver_age'] = merged_df.apply(lambda x: relativedelta(x['date'], x['date_of_birth']).years, axis=1)
merged_df.drop(['date', 'date_of_birth'], axis = 1, inplace = True)

In [65]:
# fill/drop nulls

for col in ['driver_points', 'driver_wins', 'driver_standings_pos', 'constructor_points', 
            'constructor_wins' , 'constructor_standings_pos']:
    merged_df[col].fillna(0, inplace = True)
    merged_df[col] = merged_df[col].map(lambda x: int(x))
    
merged_df.dropna(inplace = True )

In [66]:
x = merged_df['qualy_time'][0]
x = '1:02.921'
float(str(x).split(':')[1])
print(x[:8])

1:02.921


In [67]:
def format_qualifying(x):
    if ':' in x:
        return float(str(x).split(':')[1]) + (60 * float(str(x).split(':')[0])) if x != 0 else 0
    else:
        return(float(x))

In [68]:
merged_df['qualy_time'] = merged_df.qualy_time.map(lambda x: format_qualifying(x))

merged_df['qualy_time'].describe()

count    687.000000
mean      83.817259
std       14.045557
min       53.377000
25%       75.980500
50%       81.640000
75%       90.854000
max      132.909000
Name: qualy_time, dtype: float64

In [69]:
# calculate difference in qualifying times

merged_df = merged_df[merged_df['qualy_time'] != 0]
merged_df.sort_values(['season', 'round', 'grid'], inplace = True)
merged_df['time_difference'] = merged_df.groupby(['season', 'round']).qualy_time.diff()
merged_df['q_delta'] = merged_df.groupby(['season', 'round']).time_difference.cumsum().fillna(0)
merged_df.drop('time_difference', axis = 1, inplace = True)

merged_df.to_csv('data/merged.csv', index=False)
merged_df.head()

,season,round,circuit_id,warm,cold,dry,wet,cloudy,driver,nationality,...,podium,driver_points,driver_wins,driver_standings_pos,constructor_points,constructor_wins,constructor_standings_pos,qualy_time,driver_age,q_delta
0,2020,1,red_bull_ring,True,False,False,False,False,bottas,Finnish,...,1,25,1,1,37,1,1,62.939,30,0.000
19,2020,1,red_bull_ring,True,False,False,False,False,max_verstappen,Dutch,...,20,0,0,20,0,0,9,63.477,22,0.538
2,2020,1,red_bull_ring,True,False,False,False,False,norris,British,...,3,16,0,3,26,0,2,63.626,20,0.687
12,2020,1,red_bull_ring,True,False,False,False,False,albon,Thai,...,13,0,0,13,0,0,9,63.868,24,0.929
3,2020,1,red_bull_ring,True,False,False,False,False,hamilton,British,...,4,12,0,4,37,1,1,62.951,35,0.012


In [70]:
pd.get_dummies(merged_df, columns=['constructor'])

,season,round,circuit_id,warm,cold,dry,wet,cloudy,driver,nationality,...,constructor_alpine,constructor_aston_martin,constructor_ferrari,constructor_haas,constructor_mclaren,constructor_mercedes,constructor_racing_point,constructor_red_bull,constructor_renault,constructor_williams
0,2020,1,red_bull_ring,True,False,False,False,False,bottas,Finnish,...,0,0,0,0,0,1,0,0,0,0
19,2020,1,red_bull_ring,True,False,False,False,False,max_verstappen,Dutch,...,0,0,0,0,0,0,0,1,0,0
2,2020,1,red_bull_ring,True,False,False,False,False,norris,British,...,0,0,0,0,1,0,0,0,0,0
12,2020,1,red_bull_ring,True,False,False,False,False,albon,Thai,...,0,0,0,0,0,0,0,1,0,0
3,2020,1,red_bull_ring,True,False,False,False,False,hamilton,British,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
753,2021,22,yas_marina,True,False,False,False,False,vettel,German,...,0,1,0,0,0,0,0,0,0,0
758,2021,22,yas_marina,True,False,False,False,False,latifi,Canadian,...,0,0,0,0,0,0,0,0,0,1
760,2021,22,yas_marina,True,False,False,False,False,russell,British,...,0,0,0,0,0,0,0,0,0,1
761,2021,22,yas_marina,True,False,False,False,False,raikkonen,Finnish,...,0,0,0,0,0,0,0,0,0,0
